In [ ]:
# 훈련과 테스트 데이터를 가져오고 전처리
def get_train_and_test_data(mode):
    # 건물 정보 가져오기
    building_info = pd.read_csv('open/building_info.csv').drop(['ESS저장용량(kWh)', 'PCS용량(kW)'], axis=1)
    building_info = process_info(building_info)

    # 훈련 데이터 가져오기 및 전처리
    train_data = pd.read_csv('open/train.csv').drop(['일조(hr)', '일사(MJ/m2)'], axis=1)
    train_data.columns = ['num_date_time', 'building', 'date_time', 'temp', 'prec', 'wind', 'hum', 'target']
    train_data = process_data(train_data, mode)

    # 건물 정보 병합
    test_data = pd.read_csv('open/test.csv')
    test_data.columns = ['num_date_time', 'building', 'date_time', 'temp', 'prec', 'wind', 'hum']
    test_data = process_data(test_data, mode)

    # 통계치 계산
    train_data, test_data = mean_std(train_data, test_data, mode)

    if mode == 'all' or mode == 'gu_all':
        train_data = train_data.merge(building_info, on='building', how='left')
        test_data = test_data.merge(building_info, on='building', how='left')

    return train_data, test_data

def process_info(data, is_train=True):
    # 열 이름 변경
    data.columns = ['building', 'type', 'all_area', 'cool_area', 'sun']
    # sun 열의 '-'를 0으로 변경하고 float로 형변환
    data['sun'] = data['sun'].replace('-', 0).astype('float')

    # 건물 타입을 숫자로 매핑
    value_dict = {value: index for index, value in enumerate(data['type'].unique())}
    data['type'] = data['type'].map(value_dict)

    filtered_data = data[(data['type'] == 7) & (data['cool_area'] != 0)]
    result = (filtered_data['all_area'].iloc[1:].sum() / filtered_data['cool_area'].iloc[1:].sum())
    condition = (data['type'] == 7) & (data['cool_area'] == 0)
    data.loc[condition, 'cool_area'] = (data.loc[condition, 'all_area'] / result).astype('int')

    filtered_data = data[(data['type'] == 9) & (data['cool_area'] > 500)]
    result = (filtered_data['all_area'].sum() / filtered_data['cool_area'].sum())
    condition = (data['type'] == 9) & (data['cool_area'] < 500)
    data.loc[condition, 'cool_area'] = round(data.loc[condition, 'all_area'] / result, 1)

    return data

#### 코드분석
 - process_info()
     - 빌딩정보에 태양광용량(sun)결측치 0으로 처리하고 정수형 타입을 float으로 변환
     - filtered_data, type=7일때 냉방면적이 0이 아닌 데이터에 대해 전체 면적(all_area) 대비 냉방 면적의 비율을 계산
     - condition, type=9일때 냉방 면적이 500보다 큰 경우 전체 면적 대비 냉방 면적을 조정
       -  data.loc[condition, 'cool_area']: condition을 만족하는 행들의 cool_area열에 접근
       - data.loc[condition, 'all_area'] / result: 조건 만족 행들의 'all_area'열을 평균 비율인 result로 나눈 값
       - round(...,1): 소수점 첫째자리
 - get_train_and_test_data()
     - 건물정보에서 훈련데이터와 상관이 없다고 생각한 ESS, PCS는 제거한것으로 전처리에서 파악, train은 test와 맞추기 위해 일조 일사 제거
     - mode에 따라 평균 및 표준 편차 계산 혹은 다른 통계치 계산
     - if mode == 'all' or mode == 'gu_all': mode가 둘중 하나일 때 building_info를 병합
     - 다른 mode에선 건물정보 활용하지 않음

> mode는 데이터 전처리와 관련된 다양한 옵션을 의미하고 다양한 데이터 전처리 전략을 트리거하도록 설계

🎯미션 1
|Mode|전처리 차이점|장점|단점|
|---|---|---|---|
|'all'|모든 데이터에 대한 일괄처리|전체 데이터에 대한 통합된 전처리 수행|일부 특정데이터 특성 누락 가능성있음|
|'byb'|건물별로 전처리|각 건물마다 개별적인 전처리 수행해 모델 세밀 조정 가능|건물데이터 부족할 경우 모델조정에 어려움|
|'gu_all'|각 구별로 테이터를 나눈 후 구별로 전처리|각 구별로 전체 데이터를 나눠 구별로 전처리 수행|각 구별이 다른 특성을 갖는 경우 모델링에 영향|
|'gu_byb'|각 구별로 데이터를 나눈 후 건물별로 전처리|각 구별로 건물별 전처리를 수행하여 구별, 건물별 조정|각 구별이 다른 특성을 갖는 경우 모델링에 영향|


# 모델링 과정 분석

In [ ]:
# 건물 by 건물로 학습
gu_byb = ['num_date_time', 'building', 'date_time', 'temp', 'wind', 'hum',
       'dow', 'month', 'week', 'dow_hour_mean', 'holiday',
       'holiday_mean', 'holiday_std', 'hour_mean', 'hour_std', 'sin_time',
       'cos_time', 'THI', 'WC', 'CDH', 'target']

# 훈련과 테스트 데이터를 가져오고 전처리 / mode == 'gu_byb'이기에 merge가 일어나지 않음
train, test = get_train_and_test_data('gu_byb')

train = train[gu_byb]
test = test[gu_byb[:-1]] #target 제외하고 gu_byb 컬럼 선정

scores = []
best_it = []

score = pd.DataFrame({'building':range(1,101)})
for i in tqdm(range(100)):
    # 개별 건물에 대한 x, y 지정
    y = train.loc[train.building == i+1, 'target']
    x = train.loc[train.building == i+1, ].iloc[:, 3:].drop(['target'], axis=1)

    # 시계열성을 고려한 validation 구축
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)

    xgb = XGBRegressor(colsample_bytree=0.8, eta=0.01, max_depth=5,
             min_child_weight=6,n_estimators=2000, subsample=0.9, early_stopping_rounds=50, eval_metric=SMAPE)

    xgb.set_params(**{'objective':weighted_mse(100)})

    xgb.fit(x_train, y_train, eval_set=[(x_train, y_train),
                                            (x_valid, y_valid)], verbose=False)

    y_pred = xgb.predict(x_valid)

    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    best_it.append(xgb.best_iteration+1) # 각 건물들에 대한 최적 반복횟수

score['score'] = scores
print(sum(scores)/len(scores)) # 평균 점수
print(sum(best_it)/len(best_it)) # 평균 최적 반복횟수
# 4.404954637397799
# 451.67

# 정답을 담아두기 위한 배열 생성
preds = np.array([])

for i in tqdm(range(100)):
    pred_df = pd.DataFrame()

    # 각 건물에 대해서 seed 0~4인 경우를 반복해서 학습 - 추론
    for seed in [0,1,2,3,4]:
        # 건물별 train x와 y, test 구축
        y_train = train.loc[train.building == i+1, 'target']
        x_train = train.loc[train.building == i+1, ].iloc[:, 3:].drop(['target'], axis=1)
        x_test = test.loc[test.building == i+1, ].iloc[:,3:]

        # 각 건물에 대한 최적 반복횟수 불러오기
        xgb = XGBRegressor(colsample_bytree=0.8, eta=0.01, max_depth=5, seed=seed,
                 min_child_weight=6,n_estimators=best_it[i], subsample=0.9)

        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        pred_df.loc[:,seed] = y_pred # 추론 결과를 배열에 저장 (seed값이 열로 존재)

    # 배열의 같은 행에 존재하는 seed별 예측값의 평균
    pred = pred_df.mean(axis=1)

    # seed별 평균값(pred)을 preds 배열에 업데이트 (건물 0, 1, ... 99, 100 순서 즉, sample_submission 순서와 동일)
    preds = np.append(preds, pred)

submission = pd.read_csv(os.path.join(base_path,'sample_submission.csv'))
submission['answer'] = preds
submission.to_csv('./gu_byb.csv', index = False) # 건물별 예측을 gu_byb로 저장

del train, test, scores, best_it, submission

**mode=gu_byb**
코드 설명
- gu_byb mode로 get_train_and_test_data()하여 데이터 병합이 진행되지 않음.
- `train = train[gu_byb]`: gu_byb에 정의된 컬럼들만 선택해 훈련데이터로 사용
- `test = test[gu_byb[:-1]]`: gu_byb리스트에서 마지막 요소(target)를 제외한 나머지 선택

    - scores: 모델평가점수(SMAPE) 저장, best_it: 건물에 대한 최적 반복횟수
    - 타겟변수를 왜 이렇게 설정했는지 아직 의문..
    - XGBRegressor(colsample_bytree:선택할 특성비율, eta:학습률, max_depth:트리의깊이, min_child_weight: 자식트리에 필요한 최소 가중치의 합, n_estimators: 트리의수, subsample: 학습 데이터셋 사용할 비율,early_stopping_rounds:조기중지, eval_metric: 검증데이터 평가지표 나타내는 파라미터)
    - xgb.set_params(**{'objective':weighted_mse(100)}): 평균제곱 오차사용하고 가중치는 100으로 설정된 목적함수 사용
    - fit: 모델학습, eval_set파라미터 통해 검증데이터 지정하고 훈련 과정은 출력하지않음
    - predict: 예측 수행
    - SMAPE: 모델의 예측과 실제값 사이 평균 절대 백분율 오차를 나타냄
    - best_iteration은 모델이 자동으로 선택한 최적의 반복횟수

> 각 건물에 대해 반복해 해당 건물에 대한 특성과 타겟을 추출하여 모델을 학습한다. 시계열성을 고려해 검증데이터 분할하고 XGBoost회귀모델을 설정하고 학습함. 학습된 모델로 검증데이터에 대한 예측 수행하고 평가지표 계산해 성능을 평가한다

100개의 건물에 대해 반복하여 각 건물별 예측을 수행
seed값을 0~4까지 사용하여 모델을 초기화 하고 학습 -> 이를 통해 다양한 초기화 조건으로 학습하고 다양성을 활용해 강건한 예측을 수행하도록 한다.
  - y_train, x_train의 building을 왜 i+1로 처리한지는 의문, y_train은 해당건물의 target을 예측하려는 값
  - x_train: 건물(i+1)에 대한 학습데이터(특징), 필요한 열(3번째열부터 마지막 열)을 사용해 데이터 선택후 target제거후 사용
  - x_test: 필요한 열만 사용
  - 모델은 XGBoost 사용


In [ ]:
import xgboost as xgb

# 전체 건물로 학습
gu_all = ['num_date_time', 'building', 'date_time', 'temp', 'wind', 'hum',
       'type', 'all_area', 'cool_area', 'dow', 'month', 'week',
       'dow_hour_mean', 'date', 'holiday', 'holiday_mean', 'holiday_std',
       'hour_mean', 'hour_std', 'sin_time', 'cos_time', 'THI', 'WC', 'CDH', 'target']

train, test = train[gu_all], test[gu_all[:-1]]

train['date'] = pd.to_datetime(train['date'])
train['building'] = train['building'].astype('category')
train['type'] = train['type'].astype('category')

x_train = train[train['date'] < f'2022-08-18'].drop(['num_date_time', 'date_time', 'target', 'date'], axis=1).reset_index(drop=True)
x_valid = train[train['date'] >= f'2022-08-18'].drop(['num_date_time', 'date_time', 'target', 'date'], axis=1).reset_index(drop=True)
y_train = train[train['date'] < f'2022-08-18']['target'].reset_index(drop=True)
y_valid = train[train['date'] >= f'2022-08-18']['target'].reset_index(drop=True)

dtrain = xgb.DMatrix(data=x_train, label=y_train, enable_categorical=True)
dvalid = xgb.DMatrix(data=x_valid, label=y_valid, enable_categorical=True)

#df에서 각 열의 첫 값을 가져옴
param = {
    'reg_lambda': df['params_reg_lambda'][0] ,
    'gamma': df['params_gamma'][0],
    'reg_alpha': df['params_reg_alpha'][0] ,
    'colsample_bytree': df['params_colsample_bytree'][0] ,
    'subsample': df['params_subsample'][0] ,
    'max_depth': df['params_max_depth'][0],
    'min_child_weight': df['params_min_child_weight'][0],
}

model = xgb.train(params=param, dtrain=dtrain, num_boost_round=1000,
                  evals=[(dvalid, 'valid')], early_stopping_rounds=100, verbose_eval=False)

preds = model.predict(dvalid)
smape = SMAPE(y_valid, preds)

best_it = model.best_iteration+1
building_score = []

# 건물마다의 validation SMAPE score를 도출
for i in range(100):
    building_score.append(SMAPE(y_valid[i*168:(i+1)*168], preds[i*168:(i+1)*168]))

# 해당 결과는 score.csv에 저장
score['score_all'] = building_score
score.to_csv('./score.csv', index=False)
print(smape, best_it)

**mode=gu_all**
building_info로 병합된 모든 데이터들을 사용
 - 빌딩과 타입 열은 카테고리형 변수로 변환, date열은 날짜 정보로 변환
 - x_train, x_valid 특정 날짜(2022-08-18)이전과 이후로 데이터 분할(XGBoost모델 학습과 검증에 사용)
 - y_train, y_valid 특정 날짜(2022-08-18)이전과 이후의 tartget값 추출(목푯값으로 모델학습과 검증해 성능평가)
 - dtrain, dvalid: XGBoost모델에 필요한 DMatrix형식으로 데이터 변환
(왜 여기서만 DMatrix형식으로 바꿨을까? 여전히 matrix형식이였을 것 같은데?..라는 의문)
 - XGBoost모델을 학습

In [ ]:
# 건물 by 건물로 학습
# pdf p18 : V2의 3번
# 더 많은 feature를 사용, Feature 가중치 조정, 확실한 holiday만 마킹
byb = ['num_date_time', 'building', 'date_time', 'temp', 'wind', 'hum',
       'dow', 'month', 'week', 'dow_hour_mean', 'holiday',
       'holiday_mean', 'holiday_std', 'hour_mean', 'hour_std', 'sin_time',
       'cos_time', 'THI', 'WC', 'CDH', 'summer_cos', 'summer_sin', 'target']

# 훈련과 테스트 데이터를 가져오고 전처리 / mode == 'byb'이기에 merge 발생하지 않음
train, test = get_train_and_test_data('byb')
train, test = train[byb], test[byb[:-1]]

scores = []
best_it = []

for b in tqdm(range(100)):
    y = train.loc[train.building == b+1, 'target']
    x = train.loc[train.building == b+1, ].iloc[:, 3:].drop(['target'], axis=1)
    y_train, y_valid, x_train, x_valid = temporal_train_test_split(y = y, X = x, test_size = 168)

    xgb = XGBRegressor(colsample_bytree=0.8, eta=0.1, max_depth=5,
         min_child_weight=6,n_estimators=1000, subsample=0.9, early_stopping_rounds=50)

    xgb.fit(x_train, y_train, eval_set=[(x_valid, y_valid)], verbose=False)

    y_pred = xgb.predict(x_valid)


    sm = SMAPE(y_valid, y_pred)
    scores.append(sm)
    best_it.append(xgb.best_iteration+1)

print(sum(scores)/len(scores))
print(sum(best_it)/len(best_it))
# 5.17304185151095
# 107.93

# 11개 seed에 대해서 건물별 최적 반복횟수로 예측 사행
preds = np.array([])

for i in tqdm(range(100)):
    pred_df = pd.DataFrame()

    for seed in [0,1,2,3,4,5,6,7,8,9,10]:
        y_train = train.loc[train.building == i+1, 'target']
        x_train = train.loc[train.building == i+1, ].iloc[:, 3:].drop(['target'], axis=1)
        x_test = test.loc[test.building == i+1, ].iloc[:,3:]

        xgb = XGBRegressor(colsample_bytree=0.8, eta=0.1, max_depth=5, seed=seed,
             min_child_weight=6,n_estimators=best_it[i], subsample=0.9)

        xgb.fit(x_train, y_train)
        y_pred = xgb.predict(x_test)
        pred_df.loc[:,seed] = y_pred

    # 예측 결과 평균으로 앙상블
    pred = pred_df.mean(axis=1)
    preds = np.append(preds, pred)

submission = pd.read_csv('open/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('./byb.csv', index = False)
submission

del train, test, scores, best_it, submission

**mode=byb**
merge하지 않고 feature그대로 사용
 1. 건물 별 모델학습
   - 100개의 건물에 대해 반복
   - 각 건물에 대한 훈련 및 검증데이터 생성
   - XGBoost모델 사용해 학습
   - 학습 중, early_stopping_rounds을 사용해 조기 중단 조건 설정
   - 검증데이터에 대한 예측 생성후 SMAPE점수 계산

 2. 건물 별 최적 반복 횟수 예측
   - 각 건물에 대해 11개의 다른 seed값 적용하여 최적 반복횟수 예측
   - 각 seed에 대해 모델 다시 학습 후 테스트 데이터에 대한 예측 생성
 3. 앙상블 및 결과 제출
   - 11개의 예측을 평균하여 최종예측 생성

In [ ]:
import xgboost as xgb

# 전체 건물로 학습 (pdf에서 말한 것과 다르나 코드상으로는 전체 건물로 학습)
# pdf p18 : V2의 4번
# 더 많은 feature를 사용, Feature 가중치 조정, 확실한 holiday만 마킹
_all = ['num_date_time', 'building', 'date_time', 'temp', 'prec', 'wind', 'hum',
       'type', 'all_area', 'cool_area', 'sun', 'dow', 'month',
       'week', 'avg_temp', 'max_temp', 'min_temp', 'temp_diff',
       'dow_hour_mean', 'holiday', 'holiday_mean', 'holiday_std',
       'hour_mean', 'hour_std', 'sin_time', 'cos_time', 'THI', 'WC', 'CDH', 'target']

train, test = get_train_and_test_data('all')
train, test = train[_all], test[_all[:-1]]

train['date'] = pd.to_datetime(train['date_time'])
train['building'] = train['building'].astype('category')
train['type'] = train['type'].astype('category')

x_train = train[train['date'] < '2022-08-18'].drop(['num_date_time', 'date_time', 'target', 'date'], axis=1)
x_valid = train[train['date'] >= '2022-08-18'].drop(['num_date_time', 'date_time', 'target', 'date'], axis=1)
y_train = train[train['date'] < '2022-08-18']['target']
y_valid = train[train['date'] >= '2022-08-18']['target']

dtrain = xgb.DMatrix(data=x_train, label=y_train, enable_categorical=True)
dvalid = xgb.DMatrix(data=x_valid, label=y_valid, enable_categorical=True)

param = {
    'reg_lambda': df['params_reg_lambda'][0] ,
    'gamma': df['params_gamma'][0],
    'reg_alpha': df['params_reg_alpha'][0] ,
    'colsample_bytree': df['params_colsample_bytree'][0] ,
    'subsample': df['params_subsample'][0] ,
    'max_depth': df['params_max_depth'][0],
    'min_child_weight': df['params_min_child_weight'][0],
    'eta' : 0.1,
}

model = xgb.train(params=param, dtrain=dtrain, num_boost_round=1000,
                  evals=[(dvalid, 'valid')], early_stopping_rounds=50, verbose_eval=False)

preds = model.predict(dvalid)

smape = SMAPE(y_valid, preds)

score = smape
best_it = model.best_iteration+1 # 전체 건물로 학습했을 때 최적 iteration값 저장
print(score, best_it) # 4.946742493265771 [844]

preds = np.array([])

test['date'] = pd.to_datetime(test['date_time'])
test['building'] = test['building'].astype('category')
test['type'] = test['type'].astype('category')

x_train = train.drop(['num_date_time', 'date_time', 'target', 'date'], axis=1)
x_test = test.drop(['num_date_time', 'date_time', 'date'], axis=1)
y_train = train['target']

dtrain = xgb.DMatrix(data=x_train, label=y_train, enable_categorical=True)
dtest = xgb.DMatrix(data=x_test, enable_categorical=True)

pred_df = pd.DataFrame()

i = 0
for seed in tqdm(range(5)):
    # seed 0~4에 대해서 실행
    param = {
        'reg_lambda': df['params_reg_lambda'][0] ,
        'gamma': df['params_gamma'][0],
        'reg_alpha': df['params_reg_alpha'][0] ,
        'colsample_bytree': df['params_colsample_bytree'][0] ,
        'subsample': df['params_subsample'][0] ,
        'max_depth': df['params_max_depth'][0],
        'min_child_weight': df['params_min_child_weight'][0],
        'seed':seed,
        'eta':0.1
    }

    # 전에 저장해둔 최적 iteration - 100만큼 num_boost_round 실행
    model = xgb.train(params=param, dtrain=dtrain, num_boost_round=best_it-100)

    y_pred = model.predict(dtest)
    pred_df.loc[:,seed] = y_pred

pred = pred_df.mean(axis=1)
preds = np.append(preds, pred)

submission = pd.read_csv('open/sample_submission.csv')
submission['answer'] = preds
submission.to_csv('./all.csv', index = False)

del train, test
del dtrain, dtest, x_train, x_test, y_train, pred, preds

**mode==all**
2022-08-18'이전은 훈련데이터, 이후는 검증데이터로 사용
1. 데이터 전처리: 날짜와 건물정보를 카테고리 형식으로 변환
2. XGBoost모델학습: 설정한 하이퍼파라미터와 함께 학습
3. 모델평가: 검증데이터에 대한 SMAPE점수와 최적 반복 횟수로 저장

... 나머지 동일

[미션2] ratio의 역할이 왜 굳이 있었을까 라는 생각과 왜 최종 예측 결과 제출을 할때 곱해서 저장하는지 아직 이해가 잘 되지않았음

🎯미션 3

이유)
1. seed값은 알고리즘의 무작위성 제어에 사용. XGBoost와 같은 머신러닝 알고리즘은 초기화 시 무작위성을 포함하고 있어 결과가 다르게 나올 수 있다. 따라서 Randomness제어를 위해 사용되었음을 보인다.
2. 다른 시드값에서 학습한 여러 모델의 평균예측은 단일 모델의 예측보다 안정적이다.
     
기대효과)
1. 앙상블과 같은 효과를 나타냄
2. 다양한 초기화에서 얻은 결과의 평균은 모델이 다양한 특징을 학습하고 데이터의 노이즈에 민감하지 않게 만들어 모델 성능에 향상을 기대할 수 있다.

enable_categorical 매개 변수 설정이 하는 역할)

  `enable_categorical`는 XGBoost모델에서 범주형(categorical) 특징을 처리하기 위한 옵션을 설정할 때 사용.
  범주형의 특징은 문자열 또는 정수로 표시되어 있다.
  enable_categorical=True로 설정하는 것은 XGBoost에게 특정 열이 범주형임을 알려주는 역할을 한다.